   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   - -------------------------------------- 1.3/39.5 MB 7.4 MB/s eta 0:00:06
   --- ------------------------------------ 3.4/39.5 MB 8.7 MB/s eta 0:00:05
   ----- ---------------------------------- 5.2/39.5 MB 8.8 MB/s eta 0:00:04
   ------ --------------------------------- 6.8/39.5 MB 8.6 MB/s eta 0:00:04
   -------- ------------------------------- 8.4/39.5 MB 8.3 MB/s eta 0:00:04
   ---------- ----------------------------- 10.0/39.5 MB 8.0 MB/s eta 0:00:04
   ----------- ---------------------------- 11.5/39.5 MB 7.9 MB/s eta 0:00:04
   ------------- -------------------------- 13.1/39.5 MB 7.8 MB/s eta 0:00:04
   -------------- ------------------------- 14.7/39.5 MB 7.9 MB/s eta 0:00:04
   ---------------- ----------------------- 16.0/39.5 MB 7.7 MB/s eta 0:00:04
   ------------------ --------------------- 17.8/39.5 MB 7.8 MB/s eta 0:00:03
   ------------------- -------------------- 19.4/39.5 MB 7.7 MB/s eta 0:00:03

In [15]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2
import csv

print("Tensorflow version", tf.__version__)

Tensorflow version 2.19.0


In [16]:
# Cargar el modelo MoveNet desde TensorFlow Hub
model = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")

# Definir el tamaño de entrada requerido por el modelo (192x192 para la variante lightning)
input_size = 192

In [17]:
def run_inference(image):
    # Redimensionar la imagen manteniendo la proporción y añadir padding para obtener 192x192
    img = tf.image.resize_with_pad(tf.expand_dims(image, axis=0), input_size, input_size)
    
    # Convertir la imagen a int32 (tipo requerido por el modelo MoveNet)
    input_img = tf.cast(img, dtype=tf.int32)
    
    # Ejecutar la inferencia llamando directamente a la firma "serving_default"
    outputs = model.signatures['serving_default'](input_img)
    
    # La salida tiene la clave 'output_0' con forma [1, 1, 17, 3]
    keypoints = outputs['output_0'].numpy()
    return keypoints[0, 0, :, :]  # Devuelve un array de shape [17, 3]


In [18]:
# Ruta del video a procesar (modifica la ruta según tu sistema)
video_path = 'D:/la-u/ciclo 2025-1/TPI/MODELO/Video pruba/luis2.mp4'
cap = cv2.VideoCapture(video_path)

# Crear archivo CSV para guardar los keypoints
csv_file = open('keypoints_output.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['frame', 'keypoint_index', 'y', 'x', 'score'])

frame_count = 0
fps = cap.get(cv2.CAP_PROP_FPS)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convertir el frame de BGR (por defecto en OpenCV) a RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Ejecutar la inferencia para obtener los keypoints
    keypoints = run_inference(frame_rgb)
    
    # Obtener dimensiones del frame original
    height, width, _ = frame.shape
    
    # Dibujar y guardar cada keypoint
    for idx, keypoint in enumerate(keypoints):
        y, x, score = keypoint
        cx, cy = int(x * width), int(y * height)
        
        # Dibujar el keypoint si su score es mayor que el umbral (0.2)
        if score > 0.2:
            cv2.circle(frame, (cx, cy), 5, (0, 255, 0), -1)
        
        # Guardar información del keypoint en el CSV
        csv_writer.writerow([frame_count, idx, y, x, score])
    
    # Mostrar el frame con la detección de pose
    cv2.imshow('Detección de Pose con MoveNet', frame)
    frame_count += 1
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
csv_file.close()
print("Procesamiento completado.")

Procesamiento completado.
